In [1]:
import torch_bed_00_fpga_not_evol as torch_model

import numpy as np
import torch
from torchinfo import summary

# Model Configs

In [2]:
aimet_mul4 = [
    # [conv3x3/svd_conv3x3/conv1x1/svd_conv1x1/mp, (inp, mid(opt), oup)]
    ["svd_conv3x3", (3, 8, 28)],               # conv1
    ["mp"],
    ["svd_conv3x3", (28, 24, 16)],             # conv2
    ["mp"],
    ["conv1x1", (16, 16)],                     # conv31
    ["conv3x3", (16, 32)],                     # conv32
    ["conv1x1", (32, 32)],                     # conv33
    ["svd_conv3x3", (32, 52, 56)],             # conv34
    ["mp"],
    ["conv1x1", (56, 32)],                     # conv41
    ["svd_conv3x3", (32, 44, 64)],             # conv42
    ["conv1x1", (64, 32)],                     # conv43
    ["svd_conv3x3", (32, 32, 64)],             # conv44
    ["svd_conv1x1", (64, 12, 32)],             # conv45
    ["svd_conv3x3", (32, 8, 64)],              # conv46            
]

# Same name, no mul4 values
# aimet_mul4 = [
#     # [conv3x3/svd_conv3x3/conv1x1/svd_conv1x1/mp, (inp, mid(opt), oup)]
#     ["svd_conv3x3", (3, 7, 28)],               # conv1
#     ["mp"],
#     ["svd_conv3x3", (28, 22, 16)],             # conv2
#     ["mp"],
#     ["conv1x1", (16, 16)],                     # conv31
#     ["conv3x3", (16, 32)],                     # conv32
#     ["conv1x1", (32, 32)],                     # conv33
#     ["svd_conv3x3", (32, 51, 57)],             # conv34
#     ["mp"],
#     ["conv1x1", (57, 32)],                     # conv41
#     ["svd_conv3x3", (32, 44, 64)],             # conv42
#     ["conv1x1", (64, 32)],                     # conv43
#     ["svd_conv3x3", (32, 32, 64)],             # conv44
#     ["svd_conv1x1", (64, 10, 32)],             # conv45
#     ["svd_conv3x3", (32, 6, 64)],              # conv46            
# ]

In [3]:
# aimet_fpga_manual = [
#     # [conv3x3/svd_conv3x3/conv1x1/svd_conv1x1/mp, (inp, mid(opt), oup)]
#     ["conv3x3", (3, 12)],                      # conv1
#     ["mp"],
#     ["svd_conv3x3", (12, 24, 16)],             # conv2
#     ["mp"],
#     ["conv1x1", (16, 16)],                     # conv31
#     ["svd_conv3x3", (16, 16, 32)],             # conv32
#     ["conv1x1", (32, 32)],                     # conv33
#     ["svd_conv3x3", (32, 42, 64)],             # conv34
#     ["mp"],
#     ["conv1x1", (64, 30)],                     # conv41
#     ["conv3x3", (30, 60)],                     # conv42
#     ["conv1x1", (60, 32)],                     # conv43
#     ["svd_conv3x3", (32, 42, 58)],             # conv44
#     ["conv1x1", (58, 32)],                     # conv45
#     ["svd_conv3x3", (32, 20, 64)],              # conv46            
# ]

### All channels multiples of 4
aimet_fpga_manual = [
    # [conv3x3/svd_conv3x3/conv1x1/svd_conv1x1/mp, (inp, mid(opt), oup)]
    ["conv3x3", (3, 12)],                      # conv1
    ["mp"],
    ["svd_conv3x3", (12, 24, 16)],             # conv2
    ["mp"],
    ["conv1x1", (16, 16)],                     # conv31
    ["svd_conv3x3", (16, 16, 32)],             # conv32
    ["conv1x1", (32, 32)],                     # conv33
    ["svd_conv3x3", (32, 44, 64)],             # conv34
    ["mp"],
    ["conv1x1", (64, 32)],                     # conv41
    ["conv3x3", (32, 64)],                     # conv42
    ["conv1x1", (64, 32)],                     # conv43
    ["svd_conv3x3", (32, 44, 60)],             # conv44
    ["conv1x1", (60, 32)],                     # conv45
    ["svd_conv3x3", (32, 20, 64)],             # conv46            
]

# Model

In [4]:
mul_4 = torch_model.TORCH_BED_AIMET_FPGA_MUL4(model_cfg = aimet_mul4).to('cpu')

fpga = torch_model.TORCH_BED_AIMET_FPGA_MUL4(model_cfg = aimet_fpga_manual).to('cpu')

In [5]:
mul_4.eval();

fpga.eval();

## Summary

## Mul4

In [6]:
print(summary(mul_4, input_size=(1, 3, 224, 224)))

Layer (type:depth-idx)                   Output Shape              Param #
TORCH_BED_AIMET_FPGA_MUL4                [1, 2]                    --
├─Sequential: 1-1                        [1, 64, 19, 19]           --
│    └─Sequential: 2-1                   [1, 28, 222, 222]         --
│    │    └─Conv2d: 3-1                  [1, 8, 222, 224]          72
│    │    └─BatchNorm2d: 3-2             [1, 8, 222, 224]          16
│    │    └─Conv2d: 3-3                  [1, 28, 222, 222]         672
│    │    └─BatchNorm2d: 3-4             [1, 28, 222, 222]         56
│    │    └─ReLU: 3-5                    [1, 28, 222, 222]         --
│    └─MaxPool2d: 2-2                    [1, 28, 111, 111]         --
│    └─Sequential: 2-3                   [1, 16, 109, 109]         --
│    │    └─Conv2d: 3-6                  [1, 24, 109, 111]         2,016
│    │    └─BatchNorm2d: 3-7             [1, 24, 109, 111]         48
│    │    └─Conv2d: 3-8                  [1, 16, 109, 109]         1,152
│    │  

## FPGA

In [7]:
print(summary(fpga, input_size=(1, 3, 230, 230)))

Layer (type:depth-idx)                   Output Shape              Param #
TORCH_BED_AIMET_FPGA_MUL4                [1, 2]                    --
├─Sequential: 1-1                        [1, 64, 20, 20]           --
│    └─Sequential: 2-1                   [1, 12, 228, 228]         --
│    │    └─Conv2d: 3-1                  [1, 12, 228, 228]         324
│    │    └─BatchNorm2d: 3-2             [1, 12, 228, 228]         24
│    │    └─ReLU: 3-3                    [1, 12, 228, 228]         --
│    └─MaxPool2d: 2-2                    [1, 12, 114, 114]         --
│    └─Sequential: 2-3                   [1, 16, 112, 112]         --
│    │    └─Conv2d: 3-4                  [1, 24, 112, 114]         864
│    │    └─BatchNorm2d: 3-5             [1, 24, 112, 114]         48
│    │    └─Conv2d: 3-6                  [1, 16, 112, 112]         1,152
│    │    └─BatchNorm2d: 3-7             [1, 16, 112, 112]         32
│    │    └─ReLU: 3-8                    [1, 16, 112, 112]         --
│    └─Max